In [1]:
# !git clone https://github.com/hirotomusiker/cifar10_pytorch.git

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

import os
import torch
import torchvision
import tarfile
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib
import matplotlib.pyplot as plt

In [4]:
!pip install yacs

In [5]:
import os
import argparse
import torch
import os, sys
from cifar10_pytorch.cifar10_pytorch.engine import train, test
from cifar10_pytorch.cifar10_pytorch.config import cfg
from cifar10_pytorch.cifar10_pytorch.data import dataset
from cifar10_pytorch.cifar10_pytorch.modeling import build_model

In [6]:
# ! rm -r 'cifar10_pytorch'

In [7]:
# !unzip -q  '/content/gdrive/MyDrive/DLCourseProject/CINIC-10-Filtered_1K.zip' -d '/content/'

In [8]:
def arg_parser():
    parser = argparse.ArgumentParser(description="CIFAR10 training")
    parser.add_argument(
        "--config",
        default='/content/cifar10_pytorch/configs/efficientnetB0.yaml',
        help="path to config file",
        type=str
    )
    parser.add_argument(
        '--tfboard', help='tensorboard path for logging', type=str, default='out')
    parser.add_argument('--checkpoint_dir', type=str,
                        default='checkpoints',
                        help='directory where checkpoint files are saved')
    parser.add_argument('--resume', type=str,
                        default=None,
                        help='checkpoint file path')
    return parser.parse_args("")

In [9]:
!pip install tensorboardX

In [ ]:
# args and configs
args = arg_parser()
if args.config:
    cfg.merge_from_file(args.config)
cfg.freeze()
os.makedirs(args.checkpoint_dir, exist_ok=True)

# Model definition
# cfg.SOLVER.BATCHSIZE=64
model = build_model(cfg)
# cfg.MODEL.DEVICE='cpu'
print(cfg.MODEL.DEVICE)
device = cfg.MODEL.DEVICE
model.to(device)
print(1)
# Build dataloader
dataloader_train, dataloader_test = dataset.prepare_cifar10_dataset(cfg)
print(len(dataloader_train.dataset))

# Optimizer settings
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=cfg.SOLVER.BASE_LR,
                            momentum=cfg.SOLVER.MOMENTUM,
                            weight_decay=cfg.SOLVER.WEIGHT_DECAY,
                            )

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer,
    cfg.SOLVER.MILESTONES,
    cfg.SOLVER.GAMMA,
)
print(2)
if args.resume:
    checkpoint = torch.load(args.resume)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    #scheduler.last_epoch = checkpoint['epoch']
    start_epoch = checkpoint['epoch']
else:
    start_epoch = 0

# Tensorboard settings
if args.tfboard:
    print("using tfboard")
    from tensorboardX import SummaryWriter
    tblogger = SummaryWriter(args.tfboard)
else:
    tblogger = None

# Training Loop
end_epoch = cfg.SOLVER.END_EPOCH
print(3)
for epoch in range(start_epoch, end_epoch):
    scheduler.step()
    print("------ start epoch {} / {} with lr = {:.5f} -----".format(
        epoch+1, end_epoch, scheduler.get_lr()[0])
    )
    loss, train_acc = train(model, device, dataloader_train, criterion, optimizer, epoch)
    test_acc = test(model, device, dataloader_test, criterion, epoch)
    if tblogger:
        tblogger.add_scalar('train/loss', loss, epoch+1)
        tblogger.add_scalar('train/acc', train_acc, epoch+1)
        tblogger.add_scalar('test/acc', test_acc, epoch+1)
    torch.save({'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                },
                os.path.join(args.checkpoint_dir, "snapshot.ckpt"))

cpu
1



Extracting ./data/cifar-10-python.tar.gz to ./data
10100
2
using tfboard
3
------ start epoch 1 / 35 with lr = 0.01000 -----


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:417: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
